# Excercise 1 

## Introduction
In this tutorial, we will build, run and analyze a fuel cycle scenario using Jupyter Notebook, which is an python interpreter. This tutorial assumes you can run Jupyter notebooks and have already downloaded Cyclus and Cycamore.  This tutorial requires that you have a terminal, or if the user is running Windows, it is assumed they have a terminal emulator.

This tutorial will create an xml file for a once-through scenario.  The purpose of this exercise is to introduce the basic concepts of Cyclus and the layout of the user interface.  The scenario will include the following:
- a mine of natural uranium
- enrichment facility producing 4% enriched uranium oxide fuel
- LWR reactor consuming fresh fuel and producing used fuel
- repository to house all spent fuel

The first step of the tutorial will prompt the user to input values to variables based on some given parameters.

## Set Simulation Parameters

The first step in creating a Cyclus simulation will include information pertaining to the duration, start month, start year, decay treatment, and a simulation handle.

For this simulation, the following simulation parameters must be entered as prompted.

1. Duration: 480 (months)
2. Start month: 2 (February)
3. First Year: 2018
4. Decay: Never (No decay)
5. Simulation Handle:  This section will be left blank for the default handle

In [3]:
Duration = input("How long is the simulation in months? ") #months
Start_month = input("What month does the simulation begin? ") #February
Start_Year = input("What year does the simulation start? ")
Decay = input("Is there decay? ")


How long is the simulation in months? 480
What month does the simulation begin? 2
What year does the simulation start? 2018
Is there decay? Never


# Archetypes

In order to understand the capabilities of *Cyclus*, it is important to first understand archetypes.  Cyclus allows the user to switch between different models of fuel cycle facilities.  Models may differ in the way they choose to represent the physics of the facility, or in the way they choose for the facility to interact with other facilities, or both.  These models are referred to as archetypes in *Cyclus*.

### Archetype Example: Reactor Fidelity
Archetype Example: Reactor Fidelity¶

The easiest way for most fuel cycle analysts to understand the difference between archetypes is through the example of a reactor model.

- The simplest possible reactor archetype uses the concept of fixed input and output recipes to define the transmutation of material. Such a reactor might assume that all material that it receives matches its desired input recipe, and that all the material that it then ships as used fuel will match its prescribed output recipe. This will probably run the fastest.
- A more sophisticated reactor archetype uses some form of tabulated data to determine both performance characteristics and used fuel discharge composition. It could be as simple as interpolating between a set of possible input recipes and then using that interpolation on a set of corresponding output recipes. A more complex version of such an archetype could even have tabulated reactor physics paramters for more fidelity.
- The most sophisticated reactor archetype may actually be a wrapper for fuel depletion software, and perform a full depletion calculation each time new fuel arrives at the reactor. This will probably take the most computer time to complete.

Assuming that such archetypes are available, the user is welcome to choose which archetype to use, based on the modeling fidelity they are interested in and/or the performance requirements they have for their simulations.

## *Cycamore* :  The Additional *Cyclus* Module Repository

The Cyclus team has developed a simple standard set of archetypes to support basic fuel cycle modeling situations. In most cases they implement very simple models and are useful for tutorials such as this, and as a standard way to model facilities that may be at the peripheral of a problem. The Cycamore facility archetypes are:

- **Source**: This generic source of material may fill the role of any facility that produces fresh material. Depending on how much of the fuel cycle a user wants to model explicity, this could fill the role of a uranium mine, an enrichment facility, a fuel fabrication facility, etc.
- **Enrichment**: This facility archetype implements the standard equations for enrichment of U-235 in U-238, with a constrained total enrichment capacity.
- **Reactor**: This facility archetype uses simple fuel recipes for a reactor that reloads batches of assemblies at regular intervals.
- **Separations**: This facility archetype takes a number of input streams and separates all the isotopes into a number of output streams.
- **FuelFab**: This facility archetype mixes streams of fissile and fissionable material in order to best approximate a given recipe using the d-factor approach.
- **Sink**: This generic sink of material may fill the role of any facility that permanently holds used nuclear material. Depending on how much of the fuel cycle a user wants to model explicitly, this could fill the role of a geologic repository, an interim storage facility, etc.

In this tutorial, the **Source** will act as the mine, the **Enrichment** will act as the enrichment facility, the **Reactor** will act as the LWR, and the **Sink** will act as the geological repository.  The **Separations** and **FuelFab** Archetypes will not be utilized in this tutorial.  Please refer to [Example 2](http://fuelcycle.org/arche/index.html) where a more advanced simulation is built and analyzed.


## Configure an Archetype to Create a Prototype

The archetype that represents your facility can be configured with parameters that represent certain behavior.  For example, a reactor can be defined with a certain power level or certain fuel loading parameters such as cycle length and batch size.

In [4]:
#declare variables for reactor here

assem_size = 33000
n_assem_batch = 1
n_assem_core = 3
cycle_time = 11
refuel_time = 1
power_cap = 1000

# Commodity Variables

solution_priority = 1.0

# Enrichment Facility

max_feed_inventory = 1e6

#declare recipe variables here

Nat_U235_comp = 0.00711
Nat_U238_comp = 0.99289
Fresh_U235_comp = 0.04
Fresh_U238_comp = 0.96
Used_U235_comp = 0.011
Used_U238_comp = 0.94
Used_Pu239_comp = 0.009
Used_Cs137_comp = .04

In [5]:
#https://stackoverflow.com/questions/3605680/creating-a-simple-xml-file-using-python?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
import xml.etree.cElementTree as ET

## Beginning of simulation - root

root = ET.Element("simulation")

## Control Section

doc = ET.SubElement(root, "control")

ET.SubElement(doc, "duration").text = str(Duration)
ET.SubElement(doc, "startmonth").text = str(Start_month)
ET.SubElement(doc, "startyear").text = str(Start_Year)

# Archetypes Section

specs = ['cycamore']*4 +['agents']*2
specs_sub = ["Enrichment","Reactor","Source","Sink","NullRegion","NullInst"]
doc1 = ET.SubElement(root, "archetypes")
docspec1 = ET.SubElement(doc1, "spec")

lib1 = ET.SubElement(docspec1, "lib").text = (specs[0])
name1 = ET.SubElement(docspec1, "name").text = (specs_sub[0])
docspec2 = ET.SubElement(doc1, "spec")
lib2 = ET.SubElement(docspec2, "lib").text = (specs[1])
name2 = ET.SubElement(docspec2, "name").text = (specs_sub[1])
docspec3 = ET.SubElement(doc1, "spec")
lib3 = ET.SubElement(docspec3, "lib").text = (specs[2])
name3 = ET.SubElement(docspec3, "name").text = (specs_sub[2])
docspec4 = ET.SubElement(doc1, "spec")
lib4 = ET.SubElement(docspec4, "lib").text = (specs[3])
name4 = ET.SubElement(docspec4, "name").text = (specs_sub[3])
docspec5 = ET.SubElement(doc1, "spec")
lib5 = ET.SubElement(docspec5, "lib").text = (specs[4])
name5 = ET.SubElement(docspec5, "name").text = (specs_sub[4])
docspec6 = ET.SubElement(doc1, "spec")
lib6 = ET.SubElement(docspec6, "lib").text = (specs[5])
name6 = ET.SubElement(docspec6, "name").text = (specs_sub[5])

## Commodity Section

doc_commod1 = ET.SubElement(root, "commodity")

name_commod1 = ET.SubElement(doc_commod1, "name").text = "U-ore"
sol_priority1 = ET.SubElement(doc_commod1, "solution_priority").text = str(solution_priority)

doc_commod2 = ET.SubElement(root, "commodity")

name_commod2 = ET.SubElement(doc_commod2, "name").text = "Fresh-UOX-Fuel"
sol_priority2 = ET.SubElement(doc_commod2, "solution_priority").text = str(solution_priority)

doc_commod3 = ET.SubElement(root, "commodity")

name_commod3 = ET.SubElement(doc_commod3, "name").text = "Enrich-Tails"
sol_priority3 = ET.SubElement(doc_commod3, "solution_priority").text = str(solution_priority)

doc_commod4 = ET.SubElement(root, "commodity")

name_commod4 = ET.SubElement(doc_commod4, "name").text = "Used-UOX-fuel"
sol_priority4 = ET.SubElement(doc_commod4, "solution_priority").text = str(solution_priority)


#inserting enrichment facility

doc_enrich = ET.SubElement(root,"facility")

name_enrich = ET.SubElement(doc_enrich, "name").text = "EnrichPlant"
config_enrich = ET.SubElement(doc_enrich, "config")

Enrich_config = ET.SubElement(config_enrich, "Enrichment")

feedcommod_Enrich = ET.SubElement(Enrich_config, "feed_commod").text = "U-ore"
feedrecipe_Enrich = ET.SubElement(Enrich_config, "feed_recipe").text = "Nat-U"
productcommod_Enrich = ET.SubElement(Enrich_config, "product_commod").text = "Fresh-UOX-Fuel"
tailscommod_Enrich = ET.SubElement(Enrich_config, "tails_commod").text = "Enrich-Tails"
maxfeedinv_Enrich = ET.SubElement(Enrich_config, "max_feed_inventory").text = str(max_feed_inventory)

#Insert reactor

doc_ALWR = ET.SubElement(root,"facility")

name_ALWR = ET.SubElement(doc_ALWR, "name").text = "ALWR"
config_ALWR = ET.SubElement(doc_ALWR, "config")

Reactor_config = ET.SubElement(config_ALWR, "Reactor")

fuelincommod_Reactor = ET.SubElement(Reactor_config, "fuel_incommods")
val_fuelincommod = ET.SubElement(fuelincommod_Reactor, "val").text = "Fresh-UOX-Fuel"

fuelinrecipes_Reactor = ET.SubElement(Reactor_config, "fuel_inrecipes")
val_fuelinrecipes = ET.SubElement(fuelinrecipes_Reactor, "val").text = "Fresh-UOX-Fuel-4"

fueloutcommod_Reactor = ET.SubElement(Reactor_config, "fuel_outcommods")
val_fueloutcommod = ET.SubElement(fueloutcommod_Reactor, "val").text = "Used-UOX-Fuel"

fueloutrecipes_Reactor = ET.SubElement(Reactor_config, "fuel_outrecipes")
val_fueloutrecipes = ET.SubElement(fueloutrecipes_Reactor, "val").text = "Used-UOX-Fuel-4"

assemsize_Reactor = ET.SubElement(Reactor_config, "assem_size").text = str(assem_size)
nassembatch_Reactor = ET.SubElement(Reactor_config, "n_assem_batch").text = str(n_assem_batch)
nassemcore_Reactor = ET.SubElement(Reactor_config, "n_assem_core").text = str(n_assem_core)
cycletime_Reactor = ET.SubElement(Reactor_config, "cycle_time").text = str(cycle_time)
refueltime_Reactor = ET.SubElement(Reactor_config, "refuel_time").text = str(refuel_time)
powercap_Reactor = ET.SubElement(Reactor_config, "power_cap").text = str(power_cap)

## Uranium Mine Section

doc_mine1 = ET.SubElement(root, "facility")
name_Facility = ET.SubElement(doc_mine1, "name").text = "U mine"
doc_mine2 = ET.SubElement(doc_mine1, "config")

doc_mine3 = ET.SubElement(doc_mine2, "Source")
mine_outcommod = ET.SubElement(doc_mine3, "outcommod").text = "U-ore"

# Sink Section

doc_nucund = ET.SubElement(root, "facility")
doc_nucund1 = ET.SubElement(doc_nucund, "name").text = "NuclearUnderground"
config1 = ET.SubElement(doc_nucund, "config")
sink1 = ET.SubElement(config1, "Sink")
in_commods = ET.SubElement(sink1, "in_commods")
val1 = ET.SubElement(in_commods, "val").text = "Used-UOX-Fuel"

## Region Nuclandia Section

region_nuclandia = ET.SubElement(root, "region")
name_nuclandia1 = ET.SubElement(region_nuclandia, "name").text = "Nuclandia"
config_nuclandia = ET.SubElement(region_nuclandia, "config")
null_config = ET.SubElement(config_nuclandia, "NullRegion")

## institution section

institution_nuclandia = ET.SubElement(region_nuclandia, "institution")
initialfacilitylist = ET.SubElement(institution_nuclandia, "initialfacilitylist")

entry_1 = ET.SubElement(initialfacilitylist, "entry")
prototype_1 = ET.SubElement(entry_1, "prototype").text = "EnrichPlant"
number_1 = ET.SubElement(entry_1, "number").text = "1"

entry_2 = ET.SubElement(initialfacilitylist, "entry")
prototype_2 = ET.SubElement(entry_2, "prototype").text = "ALWR"
number_2 = ET.SubElement(entry_2, "number").text = "5"

entry_3 = ET.SubElement(initialfacilitylist, "entry")
prototype_3 = ET.SubElement(entry_3, "prototype").text = "U mine"
number_3 = ET.SubElement(entry_3, "number").text = "1"

entry_4 = ET.SubElement(initialfacilitylist, "entry")
prototype_4 = ET.SubElement(entry_4, "prototype").text = "NuclearUnderground"
number_4 = ET.SubElement(entry_4, "number").text = "1"

nuclear_company_name = ET.SubElement(institution_nuclandia, "name").text = "MyNucCo"

nullInst_config = ET.SubElement(institution_nuclandia, "config")
nullInst1 = ET.SubElement(nullInst_config, "NullInst")

#Insert Recipes for natural uranium, fresh UOX fuel, and used UOX fuel

#natural uranium

doc_recipeNat = ET.SubElement(root,"recipe")

name_recipeNat = ET.SubElement(doc_recipeNat, "name").text = "Nat-U"
basis_recipeNat = ET.SubElement(doc_recipeNat, "basis").text = "mass"

nuclideU235_recipeNat = ET.SubElement(doc_recipeNat, "nuclide")
id_nuclideU235 = ET.SubElement(nuclideU235_recipeNat, "id").text = "92235"
comp_nuclideU235 = ET.SubElement(nuclideU235_recipeNat, "comp").text = str(Nat_U235_comp)

nuclideU238_recipeNat = ET.SubElement(doc_recipeNat, "nuclide")
id_nuclideU238 = ET.SubElement(nuclideU238_recipeNat, "id").text = "92238"
comp_nuclideU238 = ET.SubElement(nuclideU238_recipeNat, "comp").text = str(Nat_U238_comp)

#fresh uox fuel

doc_recipeFresh = ET.SubElement(root,"recipe")

name_recipeFresh = ET.SubElement(doc_recipeFresh, "name").text = "Fresh-UOX-Fuel-4"
basis_recipeFresh = ET.SubElement(doc_recipeFresh, "basis").text = "mass"

nuclideU235_recipeFresh = ET.SubElement(doc_recipeFresh, "nuclide")
id_nuclideU235 = ET.SubElement(nuclideU235_recipeFresh, "id").text = "92235"
comp_nuclideU235 = ET.SubElement(nuclideU235_recipeFresh, "comp").text = str(Fresh_U235_comp)

nuclideU238_recipeFresh = ET.SubElement(doc_recipeFresh, "nuclide")
id_nuclideU238 = ET.SubElement(nuclideU238_recipeFresh, "id").text = "92238"
comp_nuclideU238 = ET.SubElement(nuclideU238_recipeFresh, "comp").text = str(Fresh_U238_comp)

#used uox fuel

doc_recipeUsed = ET.SubElement(root,"recipe")

name_recipeUsed = ET.SubElement(doc_recipeUsed, "name").text = "Used-UOX-Fuel-4"
basis_recipeUsed = ET.SubElement(doc_recipeUsed, "basis").text = "mass"

nuclideU235_recipeUsed = ET.SubElement(doc_recipeUsed, "nuclide")
id_nuclideU235 = ET.SubElement(nuclideU235_recipeUsed, "id").text = "92235"
comp_nuclideU235 = ET.SubElement(nuclideU235_recipeUsed, "comp").text = str(Used_U235_comp)

nuclideU238_recipeUsed = ET.SubElement(doc_recipeUsed, "nuclide")
id_nuclideU238 = ET.SubElement(nuclideU238_recipeUsed, "id").text = "92238"
comp_nuclideU238 = ET.SubElement(nuclideU238_recipeUsed, "comp").text = str(Used_U238_comp)

nuclidePu239_recipeUsed = ET.SubElement(doc_recipeUsed, "nuclide")
id_nuclidePu239 = ET.SubElement(nuclidePu239_recipeUsed, "id").text = "94239"
comp_nuclidePu239 = ET.SubElement(nuclidePu239_recipeUsed, "comp").text = str(Used_Pu239_comp)

nuclideCs137_recipeUsed = ET.SubElement(doc_recipeUsed, "nuclide")
id_nuclideCs137 = ET.SubElement(nuclideCs137_recipeUsed, "id").text = "55137"
comp_nuclideCs137 = ET.SubElement(nuclideCs137_recipeUsed, "comp").text = str(Used_Cs137_comp)


tree = ET.ElementTree(root)
tree.write("tutorial_input.xml")